# Nonlinear Optimization with Tensorflow 

## Why bother?

# Improved Genetic Algorithm

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [2]:
sess = tf.InteractiveSession()

In [3]:
# Genetic Algorithm Parameters
pop_size = 1000
features = 4
selection = 0.7
mutation = 0.02 #1. / pop_size
generations = 5000
num_parents = int(pop_size * selection)
num_children = pop_size - num_parents

In [4]:
with tf.name_scope('population'):
    population = tf.Variable(tf.random_uniform([pop_size, features], minval=-500., maxval=500), name='population')
    

In [5]:
# Initialize placeholders
with tf.name_scope('mutations_ph'):
    truth_ph = tf.placeholder(tf.float32, [None, features])
    crossover_mat_ph = tf.placeholder(tf.float32, [num_children, features])
    mutation_val_ph = tf.placeholder(tf.float32, [num_children, features])

In [6]:
# Calculate Fitness
with tf.name_scope('fitness'):
    
    with tf.name_scope('schwefel'):
        fitness = 418.9829 * features - tf.reduce_sum(
            tf.multiply(population, tf.sin(tf.sqrt(tf.abs(population)))), reduction_indices=[1])

        tf.summary.histogram('histogram', fitness)

       
    with tf.name_scope('top_individuals'):
        top_vals, top_ind = tf.nn.top_k(fitness, k=pop_size)
    

In [7]:
with tf.name_scope('best_individual'):
    # Get best individual
    best_val = tf.reduce_min(top_vals)
    best_ind = tf.arg_min(top_vals, 0)
    best_individual = tf.gather(population, best_ind)

In [8]:
with tf.name_scope('parents'):
    # Get parents
    population_sorted = tf.gather(population, tf.reverse(top_ind, [0]))
    parents = tf.slice(population_sorted, [0, 0], [num_parents, features]) 

In [9]:
with tf.name_scope('offspring'):
    # Get offspring
    # Indices to shuffle-gather parents
    rand_parent1_ix = np.random.choice(num_parents, num_children)
    rand_parent2_ix = np.random.choice(num_parents, num_children)

    # Gather parents by shuffled indices, expand back to pop_size
    with tf.name_scope('random_parent_selection'):
        rand_parent1 = tf.gather(parents, rand_parent1_ix)
        rand_parent2 = tf.gather(parents, rand_parent2_ix)
    with tf.name_scope('mutations'):
        rand_parent1_sel = tf.multiply(rand_parent1, crossover_mat_ph)
        rand_parent2_sel = tf.multiply(rand_parent2, tf.subtract(1., crossover_mat_ph))
        children_after_sel = tf.add(rand_parent1_sel, rand_parent2_sel)
    
    # Mutate offspring
    mutated_children = tf.add(children_after_sel, mutation_val_ph)

In [10]:
with tf.name_scope('next_generation'):
    # Combine children and parents into new population
    new_population = tf.concat(axis=0, values=[parents, mutated_children])
    step = tf.group(population.assign(new_population))

In [11]:
# Run through generations
#with tf.Session() as sess:
tf.set_random_seed(42)
np.random.seed(42)


# Write to Tensorboard
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('/tmp/schwefel', sess.graph)

sess.run(tf.global_variables_initializer())

if not os.path.exists('/tmp/schwefel'):
    os.makedirs('/tmp/schwefel')


top_solutions = np.empty([generations, features])
for i in range(0, generations + 1):
    # Create cross-over matrices 
    crossover_mat = np.ones(shape=[num_children, features])
    crossover_point = np.random.choice(np.arange(1, features-1, step=1), num_children)
    for pop_ix in range(num_children):
        crossover_mat[pop_ix, 0:crossover_point[pop_ix]] = 0.

    # Generate mutation probability matrices
    mutation_prob_mat = np.random.uniform(size=[num_children, features])
    mutation_values = np.random.normal(0, 0.03, size=[num_children, features])
    mutation_values[mutation_prob_mat >= mutation] = 0

    # Run GA step
    feed_dict = {crossover_mat_ph: crossover_mat,
                 mutation_val_ph: mutation_values}

    summary, _ = sess.run([merged, step], feed_dict=feed_dict)
    #summary = step.run(feed_dict, session=sess)
    
    if i % 10 == 0:
        writer.add_summary(summary, i)
        
    # Store epoch's best solution
    top_solution = sess.run(best_individual, feed_dict=feed_dict)

    #
    top_solutions = np.append(top_solutions, [top_solution], axis=0)


    #print(population.eval(session=sess))
    if i % 1000 == 0:
        best_fit = sess.run(best_val, feed_dict=feed_dict)
        print('Generation {}, Best Fitness {:.6f}'.format(i, best_fit))

# cleanup
writer.close()



Generation 0, Best Fitness 465.716431
Generation 1000, Best Fitness 4.360107
Generation 2000, Best Fitness 1.311157
Generation 3000, Best Fitness 0.052734
Generation 4000, Best Fitness 0.000000
Generation 5000, Best Fitness 0.000000


In [12]:
import pandas as pd

def schwefel(x: np.ndarray)-> np.float32 :
    return 418.9829 * len(x) - x.dot(np.sin(np.sqrt(np.abs(x))))

df = pd.DataFrame(np.round(top_solutions, decimals=4)).drop_duplicates()
df['yhat'] = df.apply(schwefel, axis=1)
df.sort_values(by='yhat')
df[-10:]

,0,1,2,3,yhat
9933,420.9520,420.9727,420.9759,420.9534,0.000124
9934,420.9682,420.9846,420.9816,420.9615,0.000110
9941,420.9677,420.9853,420.9816,420.9868,0.000148
9943,420.9720,420.9782,420.9816,420.9718,0.000086
9948,420.9675,420.9782,420.9816,420.9804,0.000100
9963,420.9684,420.9638,420.9816,420.9804,0.000092
9969,420.9720,420.9615,420.9670,420.9534,0.000089
9979,420.9541,420.9744,420.9772,420.9682,0.000091
9989,420.9684,420.9853,420.9816,420.9682,0.000106
9992,420.9680,420.9727,420.9816,420.9497,0.000120
